# Data Cleaning and Preparation

In [61]:
import io
import os
import glob
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option("display.max_columns", 100)
directory_path = '../Datasets'

survey_results_path = f'{directory_path}/survey_results_public.csv'
schema_path = f'{directory_path}/survey_results_schema.csv'

survey_df = pd.read_csv(survey_results_path)
schema_df = pd.read_csv(schema_path)


## Selecting the required columns from the dataframe
#### 15 columns are used for the analysis.

In [62]:
selected_columns = ['Age', 'Employment', 'RemoteWork', 'CodingActivities', 'EdLevel', 
                    'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'Country', 
                    'ConvertedCompYearly', 'LearnCodeCoursesCert', 'LanguageHaveWorkedWith', 
                    'DatabaseHaveWorkedWith','PlatformHaveWorkedWith']
survey_df = survey_df[selected_columns]

print(survey_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89184 entries, 0 to 89183
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     89184 non-null  object 
 1   Employment              87898 non-null  object 
 2   RemoteWork              73810 non-null  object 
 3   CodingActivities        73764 non-null  object 
 4   EdLevel                 87973 non-null  object 
 5   YearsCode               87435 non-null  object 
 6   YearsCodePro            66136 non-null  object 
 7   DevType                 76872 non-null  object 
 8   OrgSize                 65043 non-null  object 
 9   Country                 87973 non-null  object 
 10  ConvertedCompYearly     48019 non-null  float64
 11  LearnCodeCoursesCert    37076 non-null  object 
 12  LanguageHaveWorkedWith  87140 non-null  object 
 13  DatabaseHaveWorkedWith  73435 non-null  object 
 14  PlatformHaveWorkedWith  63628 non-null

## Handling Missing Values

In [63]:
print("\nMissing Values: ")
print(survey_df.isnull().sum())


Missing Values: 
Age                           0
Employment                 1286
RemoteWork                15374
CodingActivities          15420
EdLevel                    1211
YearsCode                  1749
YearsCodePro              23048
DevType                   12312
OrgSize                   24141
Country                    1211
ConvertedCompYearly       41165
LearnCodeCoursesCert      52108
LanguageHaveWorkedWith     2044
DatabaseHaveWorkedWith    15749
PlatformHaveWorkedWith    25556
dtype: int64


#### Different columns require different style of handling with missing values. Some critical columns like Yearly Compensation are required to be dropped whereas others like RemoteWork missing values can be replaced by "Not Specified" or "No Response".

In [64]:
clean_df = survey_df.dropna(subset=['ConvertedCompYearly','YearsCodePro','Country'])



In [65]:
clean_df = clean_df.assign(
    RemoteWork=clean_df['RemoteWork'].fillna('Not Specified'),
    Employment=clean_df['Employment'].fillna('Not Specified'),
    CodingActivities=clean_df['CodingActivities'].fillna('None'),
    EdLevel=clean_df['EdLevel'].fillna('Other'),
    OrgSize=clean_df['OrgSize'].fillna('Not Specified')
)


tech_columns = ['LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith']
for i in tech_columns:
    clean_df[i].fillna('None', inplace=True)
print(clean_df.isnull().sum())

Age                           0
Employment                    0
RemoteWork                    0
CodingActivities              0
EdLevel                       0
YearsCode                    25
YearsCodePro                  0
DevType                     102
OrgSize                       0
Country                       0
ConvertedCompYearly           0
LearnCodeCoursesCert      27149
LanguageHaveWorkedWith        0
DatabaseHaveWorkedWith        0
PlatformHaveWorkedWith        0
dtype: int64


/var/folders/t9/dsx6mv5s7kl5ybn8ps1lw3q40000gn/T/ipykernel_15790/108167575.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_df[i].fillna('None', inplace=True)


## Converting Data Types

In [66]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47825 entries, 1 to 89171
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     47825 non-null  object 
 1   Employment              47825 non-null  object 
 2   RemoteWork              47825 non-null  object 
 3   CodingActivities        47825 non-null  object 
 4   EdLevel                 47825 non-null  object 
 5   YearsCode               47800 non-null  object 
 6   YearsCodePro            47825 non-null  object 
 7   DevType                 47723 non-null  object 
 8   OrgSize                 47825 non-null  object 
 9   Country                 47825 non-null  object 
 10  ConvertedCompYearly     47825 non-null  float64
 11  LearnCodeCoursesCert    20676 non-null  object 
 12  LanguageHaveWorkedWith  47825 non-null  object 
 13  DatabaseHaveWorkedWith  47825 non-null  object 
 14  PlatformHaveWorkedWith  47825 non-null  obj

In [67]:
clean_df['YearsCode'] = pd.to_numeric(clean_df['YearsCode'], errors='coerce')
clean_df['YearsCodePro'] = pd.to_numeric(clean_df['YearsCodePro'], errors='coerce')
clean_df['ConvertedCompYearly'] = pd.to_numeric(clean_df['ConvertedCompYearly'], errors='coerce')

clean_df['YearsCodePro'] = clean_df['YearsCodePro'].fillna(clean_df['YearsCodePro'].median())
clean_df['YearsCode'] = clean_df['YearsCode'].fillna(clean_df['YearsCode'].median())

clean_df['DevType'] = clean_df['DevType'].fillna('Not Specified')

clean_df['LearnCodeCoursesCert'] = clean_df['LearnCodeCoursesCert'].fillna('None')

In [68]:
print(clean_df.isnull().sum())

Age                       0
Employment                0
RemoteWork                0
CodingActivities          0
EdLevel                   0
YearsCode                 0
YearsCodePro              0
DevType                   0
OrgSize                   0
Country                   0
ConvertedCompYearly       0
LearnCodeCoursesCert      0
LanguageHaveWorkedWith    0
DatabaseHaveWorkedWith    0
PlatformHaveWorkedWith    0
dtype: int64


In [69]:
clean_df.info()
clean_df.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 47825 entries, 1 to 89171
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     47825 non-null  object 
 1   Employment              47825 non-null  object 
 2   RemoteWork              47825 non-null  object 
 3   CodingActivities        47825 non-null  object 
 4   EdLevel                 47825 non-null  object 
 5   YearsCode               47825 non-null  float64
 6   YearsCodePro            47825 non-null  float64
 7   DevType                 47825 non-null  object 
 8   OrgSize                 47825 non-null  object 
 9   Country                 47825 non-null  object 
 10  ConvertedCompYearly     47825 non-null  float64
 11  LearnCodeCoursesCert    47825 non-null  object 
 12  LanguageHaveWorkedWith  47825 non-null  object 
 13  DatabaseHaveWorkedWith  47825 non-null  object 
 14  PlatformHaveWorkedWith  47825 non-null  obj

,Age,Employment,RemoteWork,CodingActivities,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,Country,ConvertedCompYearly,LearnCodeCoursesCert,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith
1,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",18.0,9.0,"Senior Executive (C-Suite, VP, etc.)",2 to 9 employees,United States of America,285000.0,Other,HTML/CSS;JavaScript;Python,Supabase,Amazon Web Services (AWS);Netlify;Vercel
2,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",27.0,23.0,"Developer, back-end","5,000 to 9,999 employees",United States of America,250000.0,None,Bash/Shell (all shells);Go,None,Amazon Web Services (AWS);Google Cloud;OpenSta...
3,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",12.0,7.0,"Developer, front-end",100 to 499 employees,United States of America,156000.0,None,Bash/Shell (all shells);HTML/CSS;JavaScript;PH...,PostgreSQL;Redis,Cloudflare;Heroku
4,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",6.0,4.0,"Developer, full-stack",20 to 99 employees,Philippines,23456.0,Other;Codecademy;edX,HTML/CSS;JavaScript;TypeScript,BigQuery;Elasticsearch;MongoDB;PostgreSQL,Amazon Web Services (AWS);Firebase;Heroku;Netl...
5,35-44 years old,"Employed, full-time",Remote,Hobby;Professional development or self-paced l...,Some college/university study without earning ...,21.0,21.0,"Developer, back-end",100 to 499 employees,United Kingdom of Great Britain and Northern I...,96828.0,Other,Bash/Shell (all shells);HTML/CSS;JavaScript;Ru...,BigQuery;Cloud Firestore;PostgreSQL;Redis,Amazon Web Services (AWS);Cloudflare;Google Cloud
6,35-44 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Profe...,Some college/university study without earning ...,4.0,3.0,"Developer, full-stack","1,000 to 4,999 employees",United States of America,135000.0,Udemy,Ada;Clojure;Elixir;Go;HTML/CSS;Java;JavaScript...,MariaDB;Microsoft SQL Server;MySQL;PostgreSQL;...,Amazon Web Services (AWS);Digital Ocean
7,25-34 years old,"Employed, full-time",Remote,Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5.0,3.0,"Developer, full-stack",10 to 19 employees,United States of America,80000.0,Codecademy;edX,Go;HTML/CSS;JavaScript;Python;Rust;SQL;TypeScript,Dynamodb;MongoDB;MySQL;Redis;SQLite,Amazon Web Services (AWS);Fly.io;Netlify;Vercel
8,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Contribute to open-source projects,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",20.0,15.0,System administrator,100 to 499 employees,Finland,64254.0,Udemy;Pluralsight,C#;JavaScript;PowerShell;Ruby;TypeScript,MariaDB;Microsoft SQL Server;MySQL;PostgreSQL;...,Digital Ocean;Microsoft Azure;Netlify
11,25-34 years old,"Employed, full-time",Remote,Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10.0,9.0,"Developer, full-stack",100 to 499 employees,Australia,78003.0,None,C#;HTML/CSS;JavaScript;Kotlin;PowerShell;Pytho...,Cosmos DB;Microsoft SQL Server;MySQL;Redis,Digital Ocean;Microsoft Azure;Netlify
12,25-34 years old,"Employed, full-time",Remote,Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10.0,9.0,"Developer, full-stack",2 to 9 employees,United States of America,75000.0,Codecademy,Bash/Shell (all shells);Elixir;HTML/CSS;JavaSc...,BigQuery;Dynamodb;Elasticsearch;MongoDB;MySQL;...,Amazon Web Services (AWS);Digital Ocean;Fly.io...


In [70]:
clean_df.reset_index(drop=True, inplace=True)

In [71]:
clean_df.head(10)

,Age,Employment,RemoteWork,CodingActivities,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,Country,ConvertedCompYearly,LearnCodeCoursesCert,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith
0,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",18.0,9.0,"Senior Executive (C-Suite, VP, etc.)",2 to 9 employees,United States of America,285000.0,Other,HTML/CSS;JavaScript;Python,Supabase,Amazon Web Services (AWS);Netlify;Vercel
1,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",27.0,23.0,"Developer, back-end","5,000 to 9,999 employees",United States of America,250000.0,None,Bash/Shell (all shells);Go,None,Amazon Web Services (AWS);Google Cloud;OpenSta...
2,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",12.0,7.0,"Developer, front-end",100 to 499 employees,United States of America,156000.0,None,Bash/Shell (all shells);HTML/CSS;JavaScript;PH...,PostgreSQL;Redis,Cloudflare;Heroku
3,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",6.0,4.0,"Developer, full-stack",20 to 99 employees,Philippines,23456.0,Other;Codecademy;edX,HTML/CSS;JavaScript;TypeScript,BigQuery;Elasticsearch;MongoDB;PostgreSQL,Amazon Web Services (AWS);Firebase;Heroku;Netl...
4,35-44 years old,"Employed, full-time",Remote,Hobby;Professional development or self-paced l...,Some college/university study without earning ...,21.0,21.0,"Developer, back-end",100 to 499 employees,United Kingdom of Great Britain and Northern I...,96828.0,Other,Bash/Shell (all shells);HTML/CSS;JavaScript;Ru...,BigQuery;Cloud Firestore;PostgreSQL;Redis,Amazon Web Services (AWS);Cloudflare;Google Cloud
5,35-44 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Profe...,Some college/university study without earning ...,4.0,3.0,"Developer, full-stack","1,000 to 4,999 employees",United States of America,135000.0,Udemy,Ada;Clojure;Elixir;Go;HTML/CSS;Java;JavaScript...,MariaDB;Microsoft SQL Server;MySQL;PostgreSQL;...,Amazon Web Services (AWS);Digital Ocean
6,25-34 years old,"Employed, full-time",Remote,Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5.0,3.0,"Developer, full-stack",10 to 19 employees,United States of America,80000.0,Codecademy;edX,Go;HTML/CSS;JavaScript;Python;Rust;SQL;TypeScript,Dynamodb;MongoDB;MySQL;Redis;SQLite,Amazon Web Services (AWS);Fly.io;Netlify;Vercel
7,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Contribute to open-source projects,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",20.0,15.0,System administrator,100 to 499 employees,Finland,64254.0,Udemy;Pluralsight,C#;JavaScript;PowerShell;Ruby;TypeScript,MariaDB;Microsoft SQL Server;MySQL;PostgreSQL;...,Digital Ocean;Microsoft Azure;Netlify
8,25-34 years old,"Employed, full-time",Remote,Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10.0,9.0,"Developer, full-stack",100 to 499 employees,Australia,78003.0,None,C#;HTML/CSS;JavaScript;Kotlin;PowerShell;Pytho...,Cosmos DB;Microsoft SQL Server;MySQL;Redis,Digital Ocean;Microsoft Azure;Netlify
9,25-34 years old,"Employed, full-time",Remote,Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10.0,9.0,"Developer, full-stack",2 to 9 employees,United States of America,75000.0,Codecademy,Bash/Shell (all shells);Elixir;HTML/CSS;JavaSc...,BigQuery;Dynamodb;Elasticsearch;MongoDB;MySQL;...,Amazon Web Services (AWS);Digital Ocean;Fly.io...


In [72]:
cleanDP = '../Datasets/cleaned_survey_results10.csv'
clean_df.head(10).to_csv(cleanDP, index = False)